# 1. Import dataset

In [56]:
import pandas as pd

### read data
data = pd.read_csv("/Users/timcerta/code/jbaccarin/xref/raw_data/gcj2008.csv")
data = data[data.flines.notna()]
data_small = data[:5]
print(data_small["flines"])

0    import java.util.Scanner;\n \n public class En...
1    import java.util.Scanner;\n \n public class Po...
2    import java.util.Scanner;\n \n public class Po...
3    import java.util.Scanner;\n \n public class No...
4    #include<cstdio>\n #include<cstdlib>\n \n #def...
Name: flines, dtype: object


# 2. Data Overview

In [42]:
data.head()

,Unnamed: 0,year,round,username,task,solution,file,full_path,flines
0,0,2008,32002,MikeSeibert,24445,0,EndlessKnight.java,gcj/2008/32002/MikeSeibert/24445/0/extracted/E...,import java.util.Scanner;\n \n public class En...
1,1,2008,32002,MikeSeibert,24444,1,Pockets.java,gcj/2008/32002/MikeSeibert/24444/1/extracted/P...,import java.util.Scanner;\n \n public class Po...
2,2,2008,32002,MikeSeibert,24444,0,Pockets.java,gcj/2008/32002/MikeSeibert/24444/0/extracted/P...,import java.util.Scanner;\n \n public class Po...
3,3,2008,32002,MikeSeibert,24446,0,NoCheating.java,gcj/2008/32002/MikeSeibert/24446/0/extracted/N...,import java.util.Scanner;\n \n public class No...
4,4,2008,32002,bhamrick,24445,0,D.cpp,gcj/2008/32002/bhamrick/24445/0/extracted/D.cpp,#include<cstdio>\n #include<cstdlib>\n \n #def...


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49072 entries, 0 to 49096
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  49072 non-null  int64 
 1   year        49072 non-null  int64 
 2   round       49072 non-null  int64 
 3   username    49072 non-null  object
 4   task        49072 non-null  int64 
 5   solution    49072 non-null  int64 
 6   file        49072 non-null  object
 7   full_path   49072 non-null  object
 8   flines      49072 non-null  object
dtypes: int64(5), object(4)
memory usage: 3.7+ MB


In [44]:
code = data.flines[0]
code

'import java.util.Scanner;\n \n public class EndlessKnight\n {\n \tpublic static void main(String[] args)\n \t{\n \t\tScanner in = new Scanner(System.in);\n \t\tint numCases = in.nextInt();\n \t\tfor (int i = 0; i < numCases; i++)\n \t\t\tdoCase(i + 1, in);\n \t}\n \n \tprivate static void doCase(int caseNum, Scanner in)\n \t{\n \t\tint H = in.nextInt();\n \t\tint W = in.nextInt();\n \t\tint R = in.nextInt();\n \t\t\n \t\tint[][] memoized = new int[W][H];\n \t\tfor (int[] arr : memoized)\n \t\t\tfor (int i = 0; i < arr.length; i++)\n \t\t\t\tarr[i] = -1;\n \t\t\n \t\tfor (int i = 0; i < R; i++)\n \t\t{\n \t\t\tint rockY = in.nextInt();\n \t\t\tint rockX = in.nextInt();\n \t\t\tmemoized[W - rockX][H - rockY] = 0;\n \t\t}\n \t\t\n \t\tint total = findMoves(W - 1, H - 1, memoized);\n \t\t\n \t\tSystem.out.println("Case #" + caseNum + ": " + total);\n \t}\n \n \tprivate static int findMoves(int i, int j, int[][] memoized)\n \t{\n \t\tif (i == 0 && j == 0)\n \t\t\treturn 1;\n \t\t\n \t\tif 

In [45]:
# Word frequencies
# different, we need to do this on the wole dataset. 
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range= (1, 2))
#X = count_vectorizer.fit_transform(data.flines).toarray()
#X

X = pd.DataFrame(count_vectorizer.fit_transform(data.flines).toarray(),
                 columns = count_vectorizer.get_feature_names_out())
X

KeyboardInterrupt: 

In [ ]:
# Calculate n-grams

def word2ngrams(text, n=3, exact=True):
    """ Convert text into character ngrams. """
    return ["".join(j) for j in zip(*[text[i:] for i in range(n)])]

ngrams = ["unigram", "bigram", "trigram"]
for count, gram in enumerate(ngrams):
    gram = word2ngrams(text = code, n = count)

# 3 Functions for data preprocessing/ feature selection

## 3.1 Create metrics

In [77]:
from statistics import mean
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from string import punctuation
from collections import Counter

def create_metrics(code):
        """
        Creates metrics needed for modeling
        :return: returns an Array with metrics calculated from the source code
        """
        # TODO: assign vars for redundant code
        # TODO: self.var for all variable assignment

        # Number of characters in code/ length of code
        code_len = len(code)

        # Number of characters in line
        code_tokens = code.split("\n")
        nchar_in_line = mean([code_len for token in code_tokens])

        # Lines of code / file length in chars
        n_lines = code.count('\n')

        # Create n-grams
        #unigram = word2ngrams(text = self.code, n = 1)
        #bigram = word2ngrams(text = self.code, n = 2)
        #trigram = word2ngrams(text = self.code, n = 3)

        # Number of words in the text / file length in characters
        word_len_ratio = len(code.split())/code_len

        # Average number of words per line / file length in characters
        pre = (code.split('\n'))
        words_list = [x.split() for i, x in enumerate(pre)]
        words_count = [len(words_list[i]) for i, x in enumerate(pre)]
        avg_words_per_line = np.mean(words_count)

        # Number of whitespace / file length in characters
        whitespace_ratio = code.count(' ')/code_len

        # Number of line breaks / file length in characters
        linebreak_ratio = code.count('\n')/len(code)

        # Number of indentations / file length in characters
        indent_ratio = code.count('\t')/code_len

        # Number of upper_case words / file length in characters
        uppercase_ratio = sum(1 for char in code if char.isupper())/code_len

        # Number of lower_case words / file length in characters
        lowercase_ratio = sum(1 for char in code if char.islower())/code_len

        # Number of punctuation symbols / file length in characters
        punctuation_count = Counter(punc for line in code for punc in line if punc in punctuation)
        punctuation_count_ratio = sum(punctuation_count.values())/ code_len

        # Average length of words(exluding punctuation, excluding single letter or number)
        code_no_punc = code
        for punc in punctuation:
            code_no_punc = code_no_punc.replace(punc, '')
        code_no_single_char = ' '.join([w for w in code_no_punc.split() if len(w)>1])
        list_len_words = [len(x) for i, x in enumerate(code_no_single_char.split())]
        avg_char_per_word = np.mean(list_len_words)

        #Keywords count ('if', 'else', 'while', 'for', 'in', 'elif', 'or', 'not', 'with', 'and', 'is')
        keyword_count_ratio = sum([code.count(x) for x in ['if', 'else', 'while', 'for', 'in', 'elif', 'or', 'not', 'with', 'and', 'is'] ]) / code_len

        # Return all metrics as array
        return np.array([code_len,
                        nchar_in_line,
                        n_lines,
                        word_len_ratio,
                        avg_words_per_line,
                        whitespace_ratio,
                        linebreak_ratio,
                        indent_ratio,
                        uppercase_ratio,
                        lowercase_ratio,
                        punctuation_count_ratio,
                        avg_char_per_word,
                        keyword_count_ratio])

In [88]:
metrics = data_small.flines.apply(lambda x:create_metrics(code = x))
metrics

0    [1118.0, 1118.0, 50.0, 0.16279069767441862, 3....
1    [2995.0, 2995.0, 152.0, 0.14156928213689482, 2...
2    [2995.0, 2995.0, 152.0, 0.14156928213689482, 2...
3    [1716.0, 1716.0, 74.0, 0.1754079254079254, 4.0...
4    [900.0, 900.0, 41.0, 0.12111111111111111, 2.59...
Name: flines, dtype: object

## 3.2 Create Count Vectorization for text + ngrams

In [46]:
def count_vectorizer(flines, analyzer = 'word', ngram_range = (1, 1), min_df = 3):
    """
    Apply count vectorizer to a given column (flines)
    :param analyzer:
        if 'word': vectorization happens with word n-grams
        if 'char': vectorization happens with character n-grams, padding the empty space to the tokens
        if 'char_wb': creates character n-grams only from text inside word boundaries
    :param ngram_range: the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted.
        examples: (1, 1) extracts unigrams, (1, 2) extracts unigrams and bigrams
    :param min_df: remove terms with frequency lower than threshold
    :return: array of count_frequencies
    """
    count_vectorizer = CountVectorizer(analyzer = analyzer, ngram_range = ngram_range)
    X = count_vectorizer.fit_transform(flines).toarray()
    return X

## Apply count_vectorizer to word

In [95]:
word_count_vec = count_vectorizer(flines = data_small["flines"])
word_count_vec

array([[ 1,  0,  0,  0,  0,  1,  3,  0,  0,  0,  0,  0,  1,  2,  0,  0,
         1,  0,  0,  0,  0,  0,  0,  2,  0,  0,  1,  0,  0,  0,  0,  4,
         0,  4,  0,  0,  0,  3,  1, 10,  0, 18,  0,  0,  1,  1,  1,  0,
         0,  0, 11,  0,  0,  2,  0,  0,  6,  0,  0,  2,  0,  0,  0,  0,
         1,  0,  0,  0,  1,  2,  0,  2,  0,  4,  0,  2,  2,  4,  0,  0,
         3,  0,  1,  0,  0,  2,  2,  0,  0,  1,  2,  0,  0,  0],
       [ 0,  0, 14, 20,  1,  1,  0,  0,  6, 16,  0,  0, 16,  2,  1,  1,
         1,  0,  0,  0,  0, 12, 11,  2,  0,  6,  0,  1,  0,  0,  0,  0,
         0, 10,  0,  0,  0,  8,  1,  8,  0, 22,  0,  0,  1,  1,  1,  0,
         0,  0,  0,  0,  0,  7,  0,  1,  3,  0,  7,  2, 10,  4,  4,  0,
         3,  4,  4,  4,  3,  1,  1,  2,  2,  0,  0,  0,  0,  4,  0,  6,
         2,  0,  2,  2,  4,  4,  0,  6,  0,  1,  2,  6,  4,  4],
       [ 0,  0, 14, 20,  1,  1,  0,  0,  6, 16,  0,  0, 16,  2,  1,  1,
         1,  0,  0,  0,  0, 12, 11,  2,  0,  6,  0,  1,  0,  0,  0,  0,
      

## Apply count_vectorizer to n-grams

In [65]:
ngram_count_vec = count_vectorizer(flines = data_small["flines"], analyzer = "char", ngram_range = (1, 3))
ngram_count_vec

array([[  1, 181,   1, ...,   0,   0,   1],
       [  1, 423,   4, ...,   0,   1,   5],
       [  1, 423,   4, ...,   0,   1,   5],
       [  1, 300,   7, ...,   1,   1,   2],
       [  1, 108,   2, ...,   0,   0,   1]])

In [87]:
np.append([metrics, word_count_vec, ngram_count_vec], values = 0)

ValueError: could not broadcast input array from shape (5,94) into shape (5,)